In [2]:
import pandas as pd
import numpy as np
import re
import string

data = pd.read_csv("../input/entity-annotated-corpus/ner_dataset.csv", encoding="latin1")

In [3]:
data = data.fillna(method="ffill")

In [4]:
df1 = pd.DataFrame({ "Sentence #":['Sentence: 47960']*6, 
                    "Word":['my', 'name', 'is', 'abhishek', 'kumar','.'],  
                    "POS":[None]*6,
                    "Tag":['O','O','O','B-per','I-per','O']})
df2 = pd.DataFrame({ "Sentence #":['Sentence: 47961']*7, 
                    "Word":['my', 'name', 'is', 'ritik', 'kumar','gupta','.'],  
                    "POS":[None]*7,
                    "Tag":['O','O','O','B-per','I-per','I-per','O']})

In [5]:
data=data.append(df1)
data=data.append(df2)

In [6]:
data.tail()

,Sentence #,Word,POS,Tag
2,Sentence: 47961,is,None,O
3,Sentence: 47961,ritik,None,B-per
4,Sentence: 47961,kumar,None,I-per
5,Sentence: 47961,gupta,None,I-per
6,Sentence: 47961,.,None,O


In [7]:
data.head(20)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [8]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words); n_words

35183

In [9]:
tags = list(set(data["Tag"].values))
n_tags = len(tags); n_tags

17

In [10]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [11]:
getter = SentenceGetter(data)

In [12]:
sent = getter.get_next()

In [13]:
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [14]:
sentences = getter.sentences

## Prepare the data
Now we introduce dictionaries of words and tags.

In [15]:
max_len = 75
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [16]:
word2idx["Obama"]

22203

In [17]:
tag2idx["B-geo"]

5

In [18]:
from keras.preprocessing.sequence import pad_sequences

# pad the sequence
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)

Using TensorFlow backend.


In [19]:
# pad the target
y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [ ]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

## Setup the CRF-LSTM
Now we can fit a LSTM-CRF network with an embedding layer.



In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
from keras.models import Model, Input,Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
import keras as k

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=n_words+1, output_dim=200, input_length=max_len))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.1)))
model.add(TimeDistributed(Dense(n_tags, activation="relu")))
crf_layer = CRF(n_tags)
model.add(crf_layer)

In [ ]:
model.summary()

In [ ]:
# adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)
model.compile(optimizer='adam', loss=crf_layer.loss_function, metrics=[crf_layer.accuracy])

In [ ]:
history = model.fit(X_tr, np.array(y_tr), batch_size=128, epochs=5,
                    validation_split=0.1, verbose=1)

In [ ]:
hist = pd.DataFrame(history.history)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure(figsize=(12,12))
plt.plot(hist["crf_viterbi_accuracy"])
plt.plot(hist["val_crf_viterbi_accuracy"])
plt.show()

In [ ]:
!pip install seqeval

In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
test_pred = model.predict(X_te, verbose=1)

In [ ]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [ ]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))


In [ ]:
print(classification_report(test_labels, pred_labels))

In [ ]:
model.evaluate(X_te, np.array(y_te))

In [ ]:
i = 1927
p = model.predict(np.array([X_te[i]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_te[i], -1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_te[i], true, p[0]):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w-1], tags[t], tags[pred]))

In [ ]:
# Custom Tokenizer
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [ ]:
test_sentence="In the context of prehistory, antiquity and contemporary indigenous peoples, the title may refer to tribal kingship. Germanic kingship is cognate with Indo-European traditions of tribal rulership (c.f. Indic rājan, Gothic reiks, and Old Irish rí, etc.)."
x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in tokenize(test_sentence)]],
                            padding="post", value=0, maxlen=max_len)

In [ ]:
p = model.predict(np.array([x_test_sent[0]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("Word", "Prediction"))
print(30 * "=")
for w, pred in zip(tokenize(test_sentence), p[0]):
    print("{:15}: {:5}".format(w, tags[pred]))